# Budget Analysis

---

## Environment Setup

In [ ]:
# Initial Imports
import os
import plaid
import requests
import datetime
import json
import pandas as pd
from dotenv import load_dotenv

%matplotlib inline

In [ ]:
# Helper functions

def pretty_print_response(response):
    """
    This function formats a JSON file to pretty printing it in the screen.
    """
    print(json.dumps(response, indent=4, sort_keys=True))

In [ ]:
# Set environment variables from the .env file
load_dotenv()

In [ ]:
# Extract API keys from environment variables
PLAID_CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
PLAID_SBX_SECRET = os.getenv("PLAID_SBX_SECRET_KEY")
PLAID_PUBLIC_KEY = os.getenv("PLAID_PUBLIC_KEY")

In [ ]:
PLAID_CLIENT_ID

In [ ]:
PLAID_SBX_SECRET

In [ ]:
PLAID_PUBLIC_KEY

---

## Plaid Access Token

In this section, you will use the `plaid-python` API to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:

1. Create a client to connect to plaid

2. Use the client to generate a public token and request the following items: ['transactions', 'income', 'assets']

3. Exchange the public token for an access token

4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [ ]:
# Create the client object
client = plaid.Client(client_id = PLAID_CLIENT_ID,
                      secret = PLAID_SBX_SECRET,
                      public_key = PLAID_PUBLIC_KEY,
                      environment = 'sandbox')

In [ ]:
# Set the institution id
INSTITUTION_ID = "ins_109508"

### 2. Generate a public token

In [ ]:
# Create the public token
create_tkn_response = client.Sandbox.public_token.create(INSTITUTION_ID,
                                                        ['transactions', 'income', 'assets'])
# Print the public token
pretty_print_response(create_tkn_response)

### 3. Exchange the public token for an access token

In [ ]:
# Retrieve the access token
exchange_response = client.Item.public_token.exchange(create_tkn_response['public_token'])

# Print the access toke
pretty_print_response(exchange_response)

In [ ]:
# Store the access token in a Python variable
access_token = exchange_response['access_token']

### 4. Fetch Accounts

In [ ]:
# Retrieve accounts
accounts = client.Accounts.get(access_token)
# Print accounts
pretty_print_response(accounts)

---

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days

2. Print the categories for each transaction type

3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)

4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [ ]:
# Set the start and end date 90 days from today
start_date = "{:%Y-%m-%d}".format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = "{:%Y-%m-%d}".format(datetime.datetime.now())
# Retrieve the transactions for the last 90 days
transactions_response = client.Transactions.get(access_token, start_date, end_date)
# Print the transactions
pretty_print_response(transactions_response)

### 2. Print the categories for each transaction

In [ ]:
# Create a for-loop to print the categories for each transaction
for transactions in transactions_response['transactions']:
    print(json.dumps(transactions['category']))

### 3. Create a new DataFrame using the following fields from the JSON transaction data: `date`, `name`, `amount`, `category`. 

(For categories with more than one label, just use the first category label in the list)

In [ ]:
# Define an empty DataFrame and set the columns as date, name, amount, and category
columns = ['date', 'name', 'amount', 'category']
# Retrieve all the transactions
all_transactions_df = pd.DataFrame(transactions_response['transactions'])
# Populate the transactions DataFrame with the transactions data
selected_df = all_transactions_df[columns]
# Display sample data from the DataFrame    
selected_df

### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [ ]:
# Display data types
selected_df.dtypes

In [ ]:
# Make any required data type transformation
selected_df['date'] = pd.to_datetime(selected_df['date'], infer_datetime_format=True)

In [ ]:
selected_df

In [ ]:
# Set the date column as index
selected_df.set_index('date', inplace=True)
# Display sample data
selected_df

---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:

1. Determine the previous year's gross income and print the results

2. Determine the current monthly income and print the results

3. Determine the projected yearly income and print the results

In [ ]:
# Fetch de income data
income_response = client.Income.get(access_token)
# Print the income data
pretty_print_response(income_response)

In [ ]:
# Determine the previous year's gross income and print the results
previous_yr_gross = income_response['income']['last_year_income']
print(f"The previous year'ss gross income is: {previous_yr_gross}")

In [ ]:
# Determine the current monthly income and print the results
current_monthly_income = income_response['income']['income_streams'][0]['monthly_income']
print(f"The current monthly income is: {current_monthly_income}")

In [ ]:
# Determine the projected yearly income and print the results
projected_yr_income = income_response['income']['projected_yearly_income']
print(f"The projected yearly income is: {projected_yr_income}")

---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)

2. Generate a bar chart with the number of transactions for each category

3. Calulate the expenses per month

4. Plot the total expenses per month

### Calculate the expenses per category

In [ ]:
arr = selected_df['category'].values
lst = [item[0] for item in arr]

In [ ]:
selected_df['category'] = lst
selected_df

In [ ]:
# Compute the total expenses per category
selected_df['category'] = lst
cat_expense_df = selected_df.groupby('category').sum()
# Display the expenses per catefory
cat_expense_df

In [ ]:
cat_expense_s = cat_expense_df['amount']

In [ ]:
# Create a spending categories pie chart.
cat_expense_s.plot(kind='pie')

### Calculate the expenses per month

In [ ]:
# Create a DataFrame with the total expenses
total_df = selected_df[['date', 'amount']]
# Display sample data
total_df

In [ ]:
total_df['date'] = pd.to_datetime(total_df['date'], infer_datetime_format=True)

In [ ]:
# Create a spending per month bar chart
total_df.set_index('date', inplace=True)

In [ ]:
total_df

In [ ]:
month_spend_grp = total_df.groupby([total_df.index.year, total_df.index.month]).sum()

In [ ]:
month_spend_grp

In [ ]:
ax = month_spend_grp.plot(kind = 'bar', title = 'Spending per month')
ax.set_xlabel('Month')
ax.set_ylabel('Amount')